In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json

In [12]:
#!pip install yahoofinancials

In [2]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    display(df3)
    filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    print(filename)
    #df3.to_csv('{}.csv'.format(filename))
    return df3
#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [270]:
ticker = 'SQ'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')

In [321]:
cash_balance = Sheet_Data(ticker,cash)
income_balance = Sheet_Data(ticker,income)
balance_balance = Sheet_Data(ticker,balance)

LMT,2017,2018,2019,2020
changeToLiabilities,164000000,377000000,-535000000,119000000
totalCashflowsFromInvestingActivities,-1147000000,-1075000000,-1241000000,-2010000000
netBorrowings,-150000000,-150000000,-1500000000,-519000000
totalCashFromFinancingActivities,-4305000000,-4152000000,-5328000000,-4527000000
changeToOperatingActivities,1316000000,-3574000000,81000000,-1197000000
netIncome,1963000000,5046000000,6230000000,6833000000
changeInCash,1024000000,-2089000000,742000000,1646000000
repurchaseOfStock,-2001000000,-1492000000,-1200000000,-1100000000
totalCashFromOperatingActivities,6476000000,3138000000,7311000000,8183000000
depreciation,1072000000,1055000000,1078000000,1124000000


LMT_cashflowStatementHistory


LMT,2017,2018,2019,2020
researchDevelopment,None,None,None,None
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,5246000000,5838000000,7241000000,8235000000
minorityInterest,74000000,55000000,44000000,23000000
netIncome,1963000000,5046000000,6230000000,6833000000
sellingGeneralAdministrative,746000000,777000000,608000000,-241000000
grossProfit,6371000000,7370000000,8367000000,8681000000
ebit,5593000000,6644000000,7698000000,8877000000
operatingIncome,5593000000,6644000000,7698000000,8877000000
otherOperatingExpenses,32000000,-51000000,61000000,45000000


LMT_incomeStatementHistory


LMT,2017,2018,2019,2020
intangibleAssets,4.221000e+09,3.941000e+09,3.724000e+09,3698000000
totalLiab,4.739600e+10,4.342700e+10,4.435700e+10,44672000000
totalStockholderEquity,-8.500000e+08,1.394000e+09,3.127000e+09,6015000000
minorityInterest,7.400000e+07,5.500000e+07,4.400000e+07,23000000
otherCurrentLiab,8.911000e+09,8.374000e+09,8.975000e+09,9116000000
totalAssets,4.662000e+10,4.487600e+10,4.752800e+10,50710000000
commonStock,2.840000e+08,2.810000e+08,2.800000e+08,279000000
otherCurrentAssets,1.509000e+09,4.180000e+08,5.310000e+08,1150000000
retainedEarnings,1.140500e+10,1.543400e+10,1.840100e+10,21636000000
otherLiab,2.097000e+10,1.642500e+10,1.898100e+10,18229000000


LMT_balanceSheetHistory


# Anaylsis

In [326]:
FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
Revenue = income_balance.loc[['totalRevenue','netIncome']]
Growth_Rate = Revenue.pct_change().mean()
#FCF is CFO - CAPEX = FCF
#FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

df = pd.DataFrame()
df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
df['Net Income'] = FCF.loc['netIncome']
#df['FCF/Net Income'] = df['FCF']/df['Net Income']
minimum = (df['FCF']/df['Net Income']).min()



average_factor = cash_balance.pct_change(axis = 1 ).mean(axis = 1 )
average_factor = (100 + average_factor)/100
df = cash_balance
df['2021'] = cash_balance[cash_balance.columns[3]] * average_factor
for i in range(0,4,1):
    num =  int(df.columns[4]) + i

    df['{}'.format(num)] = df['{}'.format(num - 1)] * average_factor

df

In [211]:
#Anaylist esitmates 
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL')
df = df[1] 
df.index = df[df.columns[0]]
df1 = df.index 
df1 = df[df.columns[3:5]]
df1 = df1.drop(['Sales Growth (year/est)'])

for i in df1.columns:
    df1[i] = df1[i].str.replace('B','e9')
    df1[i] = df1[i].astype(float)

#df1.loc['Avg. Estimate']
df1.loc['Avg. Estimate'].tolist()

Revenue1 = Revenue.loc['totalRevenue']
strs = int(Revenue.columns[3])
Revenue1['{}'.format(strs+1)] = df1.loc['Avg. Estimate'][0]
Revenue1['{}'.format(strs+2)] = df1.loc['Avg. Estimate'][1]
growth = 1 + Revenue.T.pct_change().sum()/5

Revenue1['{}'.format(strs+3)] = l.drop_index() * growth
Revenue1['{}'.format(strs+4)] = l2.drop_index() * growth
Revenue.loc['netIncome']/Revenue.loc['totalRevenue']

AttributeError: 'numpy.float64' object has no attribute 'drop_index'

In [330]:
outstanding_shares = stock['sharesOutstanding']

Required_rate = .075
Pre_rate = .025
Cashflowgrowthrate = .03

years = [1,2,3,4]
freecash_flow = df['FCF'].values

terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

futurefreecashflow = []
discountfactor = []
discountfuturefreecashflow = []
for  year in years:
    cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
    futurefreecashflow.append(cashflow)
    discountfactor.append((1+ Required_rate)**year)


for i in range(0,len(years)):
    discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

#print(discountfuturefreecashflow)
dis = terminalvalue/(1+Required_rate)**len(years)
discountfuturefreecashflow.append(dis)
value = sum(discountfuturefreecashflow)
fairvalue = value/outstanding_shares
print(fairvalue)
print(yahoo_financials.get_current_price())
#print(stock.get_current_price())

434.60495112604264
341.69


In [320]:
ticker = 'LMT'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

In [286]:
sample = pd.DataFrame()
sample['Free Cash Flow'] = df['FCF']
sample['Cash Flow'] = futurefreecashflow
sample['discountfactor'] = discountfactor
sample

,Free Cash Flow,Cash Flow,discountfactor
SQ,,,
2017,101614000,2.504970e+08,1.070000
2018,233877000,2.580119e+08,1.144900
2019,403201000,2.657523e+08,1.225043
2020,243201000,2.737249e+08,1.310796


In [324]:
df = pd.DataFrame.from_dict({(i): stock[i] 
                            for i in stock.keys() 
                            },
                        orient='index')
df

,0
annualHoldingsTurnover,None
enterpriseToRevenue,1.639
beta3Year,None
profitMargins,0.10448
enterpriseToEbitda,10.72
52WeekChange,-0.098986
morningStarRiskRating,None
forwardEps,28.04
revenueQuarterlyGrowth,None
sharesOutstanding,279784000


In [319]:
aapl.info['sharesOutstanding']
stock.keys()

dict_keys(['annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'profitMargins', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 'totalAssets', 'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 'heldPercentInsiders', 'nextFiscalYearEnd', 'yield', 'mostRecentQuarter', 'shortRatio', 'sharesShortPreviousMonthDate', 'floatShares', 'beta', 'enterpriseValue', 'priceHint', 'threeYearAverageReturn', 'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 'morningStarOverallRating', 'earningsQuarterlyGrowth', 'priceToSalesTrailing12Months', 'dateShortInterest', 'pegRatio', 'ytdReturn', 'forwardPE', 'maxAge', 'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 'impliedSharesOutstandin

In [318]:
pd.DataFrame.from_dict({(i): stock[i] 
for i in stock.keys()},orient='index')

,0
annualHoldingsTurnover,None
enterpriseToRevenue,10.685
beta3Year,None
profitMargins,0.17534
enterpriseToEbitda,107.201
52WeekChange,0.396863
morningStarRiskRating,None
forwardEps,3.5
revenueQuarterlyGrowth,None
sharesOutstanding,917734976


# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)